# Linear Regression Quiz
Use this Jupyter notebook to find the answer to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF, StringIndexer
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.sql.functions import udf, sum as Fsum, pow as Fpow, col, sqrt as Fsqrt
from pyspark.sql.types import IntegerType, FloatType
import numpy as np
import re
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [5]:
df.take(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php')]

### Build Description Length Features

In [5]:
df = df.withColumn("Desc", F.concat(F.col("Title"), F.lit(' '), F.col("Body")))

In [6]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)
body_length = udf(lambda x: len(x), IntegerType())
split_text = udf(lambda x: x.split(" "), IntegerType())
df = df.withColumn("DescLength", body_length(df.words))
df = df.withColumn("TagLength", body_length(split_text(df.Tags)))

In [8]:
df.select(["words", "DescLength", "TagLength"]).show()

+--------------------+----------+---------+
|               words|DescLength|TagLength|
+--------------------+----------+---------+
|[how, to, check, ...|        96|        5|
|[how, can, i, pre...|        83|        1|
|[r, error, invali...|      3168|        3|
|[how, do, i, repl...|       124|        3|
|[how, to, modify,...|       154|        3|
|[setting, proxy, ...|        75|        3|
|[how, to, draw, b...|       121|        1|
|[how, to, fetch, ...|       170|        3|
|[net, library, fo...|       107|        3|
|[sql, server, pro...|        74|        5|
|[how, do, commerc...|       145|        3|
|[crappy, random, ...|       148|        3|
|[migrate, from, m...|        24|        3|
|[where, can, i, f...|        49|        3|
|[can, i, stop, wi...|        48|        1|
|[php, framework, ...|       389|        3|
|[what, creates, r...|       380|        2|
|[wpf, multiple, c...|       216|        2|
|[play, framework,...|       123|        4|
|[creating, a, rep...|       404

In [9]:
df.select("TagLength").dropDuplicates().show()

+---------+
|TagLength|
+---------+
|        1|
|        3|
|        5|
|        4|
|        2|
+---------+



In [10]:
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)
df.select(["words", "DescLength", "DescVec", "TagLength"]).show()

+--------------------+----------+--------+---------+
|               words|DescLength| DescVec|TagLength|
+--------------------+----------+--------+---------+
|[how, to, check, ...|        96|  [96.0]|        5|
|[how, can, i, pre...|        83|  [83.0]|        1|
|[r, error, invali...|      3168|[3168.0]|        3|
|[how, do, i, repl...|       124| [124.0]|        3|
|[how, to, modify,...|       154| [154.0]|        3|
|[setting, proxy, ...|        75|  [75.0]|        3|
|[how, to, draw, b...|       121| [121.0]|        1|
|[how, to, fetch, ...|       170| [170.0]|        3|
|[net, library, fo...|       107| [107.0]|        3|
|[sql, server, pro...|        74|  [74.0]|        5|
|[how, do, commerc...|       145| [145.0]|        3|
|[crappy, random, ...|       148| [148.0]|        3|
|[migrate, from, m...|        24|  [24.0]|        3|
|[where, can, i, f...|        49|  [49.0]|        3|
|[can, i, stop, wi...|        48|  [48.0]|        1|
|[php, framework, ...|       389| [389.0]|    

In [11]:
import matplotlib.pyplot as plt
plt.scatter(df.select("DescLength").collect(), df.select("TagLength").collect())
plt.show()

# Question
Build a linear regression model using the length of the combined Title + Body fields. What is the value of r^2 when fitting a model with `maxIter=5, regParam=0.0, fitIntercept=False, solver="normal"`?

In [12]:
data = df.select(col("DescVec").alias("features"), col("TagLength").alias("label"))

In [13]:
data.show()

+--------+-----+
|features|label|
+--------+-----+
|  [96.0]|    5|
|  [83.0]|    1|
|[3168.0]|    3|
| [124.0]|    3|
| [154.0]|    3|
|  [75.0]|    3|
| [121.0]|    1|
| [170.0]|    3|
| [107.0]|    3|
|  [74.0]|    5|
| [145.0]|    3|
| [148.0]|    3|
|  [24.0]|    3|
|  [49.0]|    3|
|  [48.0]|    1|
| [389.0]|    3|
| [380.0]|    2|
| [216.0]|    2|
| [123.0]|    4|
| [404.0]|    5|
+--------+-----+
only showing top 20 rows



In [14]:
lr = LinearRegression(predictionCol='prediction', maxIter=5, regParam=0.0, fitIntercept=False, solver="normal")

In [15]:
#LinearRegression(featuresCol='features', labelCol='label'
model = lr.fit(data)

In [16]:
model.coefficients

DenseVector([0.0079])

In [17]:
model_summary = model.summary

In [18]:
model_summary.r2

0.4455149596308462

In [19]:
results = model.transform(data)

In [20]:
results.persist()

DataFrame[features: vector, label: int, prediction: double]

In [21]:
results.show()

+--------+-----+-------------------+
|features|label|         prediction|
+--------+-----+-------------------+
|  [96.0]|    5| 0.7605447771087013|
|  [83.0]|    1| 0.6575543385418979|
|[3168.0]|    3|  25.09797764458714|
| [124.0]|    3| 0.9823703370987391|
| [154.0]|    3| 1.2200405799452083|
|  [75.0]|    3| 0.5941756071161728|
| [121.0]|    1| 0.9586033128140922|
| [170.0]|    3| 1.3467980427966584|
| [107.0]|    3| 0.8476905328190733|
|  [74.0]|    5| 0.5862532656879572|
| [145.0]|    3| 1.1487395070912674|
| [148.0]|    3| 1.1725065313759144|
|  [24.0]|    3|0.19013619427717532|
|  [49.0]|    3|0.38819472998256627|
|  [48.0]|    1|0.38027238855435064|
| [389.0]|    3|  3.081790815575883|
| [380.0]|    2| 3.0104897427219424|
| [216.0]|    2| 1.7112257484945776|
| [123.0]|    4| 0.9744479956705234|
| [404.0]|    5| 3.2006259369991175|
+--------+-----+-------------------+
only showing top 20 rows



In [22]:
# lr2 = LinearRegression(maxIter=5, regParam=0.0, fitIntercept=False, solver="normal")
# data2 = df.select(col("DescLength").alias("features"), col("TagLength").alias("label"))
# model2 = lr.fit(data2)
# 'requirement failed: Column features must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually IntegerType.'

# logistic

In [7]:
cv = CountVectorizer(inputCol="words", outputCol="TF", vocabSize=1000)
cvmodel = cv.fit(df)
df = cvmodel.transform(df)

In [8]:
idf = IDF(inputCol="TF", outputCol="TFIDF")
idfModel = idf.fit(df)
df = idfModel.transform(df)
#df.head()

In [9]:
indexer = StringIndexer(inputCol="oneTag", outputCol="label_")
df = indexer.fit(df).transform(df)

In [26]:
#df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string, Desc: string, words: array<string>, DescLength: int, TagLength: int, DescVec: vector, TF: vector, TFIDF: vector, label_: double]

In [10]:
data2 = df.select(col("TFIDF").alias("features"), col("label_").alias("label"))

In [28]:
#data2.persist()

DataFrame[features: vector, label: double]

In [11]:
#data2.head()

In [12]:
lr2 = LogisticRegression(maxIter=10, regParam=0.0)

In [13]:
lrmodel2 = lr2.fit(data2)

In [14]:
lrmodel2.summary.accuracy

0.39868

In [16]:
results = lrmodel2.transform(data2)

In [19]:
results.select(["label", "prediction"]).show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  3.0|       3.0|
| 91.0|      91.0|
| 29.0|      29.0|
|  1.0|       1.0|
|  3.0|       5.0|
|149.0|       1.0|
|  0.0|       7.0|
|  1.0|       5.0|
|  4.0|       4.0|
| 14.0|       1.0|
+-----+----------+
only showing top 10 rows



In [ ]:
# if run all head/show commands, will lack memory and get , so to run logistic, did not run any head/show/collect
# An error occurred while trying to connect to the Java server (127.0.0.1:39543)